<a href="https://colab.research.google.com/github/yoyoyo-yo/DeepLearningMugenKnock/blob/master/pytorch/Word2vec_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2vec

# import

In [0]:
import os
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F

# Ginza

In [4]:
!pip install ginza

     |████████████████████████████████| 54.9MB 60kB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 44.8MB 69kB/s 
     |████████████████████████████████| 481kB 42.5MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=2047b11dad29d16b2b3baaa0b487ba4d0e1d2c6c1142b83e9c95fe645770b84f
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=fba8d6df290d93811d048526ff44b2ed591798d5ce0451316da2a28c6311dbbb
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for SudachiPy: filename=SudachiPy-0.4.6-cp36-cp36m-linux_x86_64.whl size=865416 sha256=74cc74fbc844734aa46e174e0ef3345a59bbecf65945b022139e39f084413234
  Stored in directory: /root/.cache/pip/wheels/83/80/5f/cc3ed568c44648e51b5984f78fa0c5015af

In [0]:
import pkg_resources, imp
imp.reload(pkg_resources)

import spacy
nlp = spacy.load('ja_ginza')

# Get corpus

In [0]:
def get_corpus(fname):
    corpus = []

    with open(fname, 'r') as f:
        for line in f.readlines():
            line = line.rstrip()
            _corpus = []
            for sent in nlp(line).sents:
                for token in sent:
                    _corpus.append(token.orth_)

            corpus = list(set(corpus) | set(_corpus))
    corpus.sort()
    return corpus

In [0]:
# sample
corpus = get_corpus('/content/drive/My Drive/Colab Notebooks/datasets/sandwitchman/sandwitchman_hanayome_original.txt')
corpus = ['<UNKNOWN>'] + corpus

In [0]:
corpus = []
for fpath in glob('/content/drive/My Drive/Colab Notebooks/datasets/sandwitchman/*_original.txt'):
    _corpus = get_corpus(fpath)
    corpus = list(set(corpus) | set(_corpus))

corpus.sort()

corpus = ['<UNKNOWN>'] + corpus

In [46]:
# show sample
corpus[:20]

['<UNKNOWN>',
 '!',
 '(',
 ')',
 '.co.jp',
 '1',
 '10',
 '100',
 '10万',
 '110',
 '12',
 '146',
 '2',
 '20',
 '200',
 '2007',
 '2008',
 '2009',
 '2010',
 '211']

In [47]:
# corpus len
len(corpus)

2337

# Get data

In [0]:
def read_data(fname, corpus):
    Xs = []
    with open(fname, 'r') as f:
        for line in f.readlines():
            line = line.rstrip()
            _Xs = [corpus.index('<SOS>')]
            for sent in nlp(line).sents:
                for token in sent:
                    w = token.orth_

                    if w in corpus:
                        ind = corpus.index(w)
                    else:
                        ind = corpus.index('<UNKNOWN>')
                    _Xs.append(ind)
            _Xs.append(corpus.index('<EOS>'))
            Xs.append(_Xs)

    return np.array(Xs)

In [0]:
def read_data(fname, corpus):
    Xs = []
    with open(fname, 'r') as f:
        data = f.read().rstrip()

    Xs = [corpus.index(token.orth_) if token.orth_ in corpus else corpus.index('<UNKNOWN>') for sent in nlp(data).sents for token in sent]

    return np.array(Xs)

In [0]:
# read_data('/content/drive/My Drive/Colab Notebooks/datasets/sandwitchman/sandwitchman_hanayome_original.txt', corpus)

In [0]:
def get_data(data, w2v_c=None):
    Xs = []
    ts = []

    w2v_x_len = 2 * w2v_c + 1

    for i in range(0, len(data) - w2v_x_len):
        Xs.append(data[i + w2v_c])
        ts.append(data[i : i + w2v_x_len])

    return Xs, ts


In [0]:
# get training data
data_Xs = []
data_ts = []

W2V_C = 3  # word2vec window size satisfying C >= 1
W2V_X_LEN = 2 * W2V_C + 1  # training label length

for fpath in glob('/content/drive/My Drive/Colab Notebooks/datasets/sandwitchman/*_original.txt'):
    data = read_data(fpath, corpus)
    _data_Xs, _data_ts = get_data(data, w2v_c=W2V_C)
    data_Xs += _data_Xs
    data_ts += _data_ts

data_Xs = np.array(data_Xs)
data_ts = np.array(data_ts)

In [78]:
# show sample
data_Xs[:3]

array([ 566,   58, 2252])

In [69]:
# data shape
data_Xs.shape, data_ts.shape

((16076, 7), (16076,))

# Model

In [0]:
class Word2Vec(torch.nn.Module):
    def __init__(self, input_dim, dim=512):
        super(Word2Vec, self).__init__()

        self.embed = torch.nn.Linear(input_dim, dim)
        self.outs = []
        for _ in range(C * 2):
            self.outs.append(torch.nn.Linear(dim, input_dim))
        self.out = torch.nn.Linear(dim, input_dim)

    def forward(self, input):
        embed = self.embed(input)

        xs = []
        for i in range(C * 2):
            x = self.outs[i](embed)
            x = F.softmax(x, dim=1)
            xs.append(x)
        #x = self.out(embed)
        #x = F.softmax(x, dim=1)
        return xs

    def get_vec(self, input):
        return self.embed(input)

In [0]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [75]:
DEVICE

'cuda'

# Utility

In [0]:
class MInibatch_Generator():
    def __init__(self, data_size, batch_size, shuffle=True):
        self.data_size = data_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.mbi = 0 # index for iteration
        self.inds = np.arange(data_size)
        np.random.shuffle(self.inds)

    def __call__(self):
        if self.mbi + self.batch_size > self.data_size:
            inds = self.inds[self.mbi:]
            if self.shuffle:
                np.random.shuffle(self.inds)
            inds = np.hstack((inds, self.inds[ : (self.batch_size - (self.data_size - self.mbi))]))
            self.mbi = self.batch_size - (self.data_size - self.mbi)
        else:
            inds = self.inds[self.mbi : self.mbi + self.batch_size]
            self.mbi += self.batch_size
        return inds

In [0]:
# train
def train():
    # model
    model = Word2Vec(voca_num, dim=hidden_dim).to(device)

    # minibatch index
    mb_gen = Minibatch_Generator(len(data_ts), batch_size=32)
    mb_gen.train()

    # loss function
    loss_fn = torch.nn.NLLLoss()
    
    # each learning rate and iteration

    # optimizer
    opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    #optimizer = torch.optim.Adam(model.parameters(), lr=lr)

        
    # each iteration
    for ite in range(ite):
        # get minibatch
        mb_inds = mb_gen()
        Xs = data_Xs[mb_inds]
        ts = data_ts[mb_inds]

        opt.zero_grad()

        ts = model(Xs)

        ts = 

        loss = 0.
        accuracy = 0.
        total_len = 0

        # each data of minibatch
        for mb_index in range(mb):
            # 1 data of minibatch
            Xs = np.array(X_inds[mb_index]).reshape([-1, 1])

            input_X = np.zeros([1, voca_num])
            input_X[:, Xs[C]] = 1
            input_X = torch.tensor(input_X, dtype=torch.float).to(device)
            
            # reset graph
            optimizer.zero_grad()
        
            # data length
            total_len += x_length

            # forward network
            ys = model(input_X)

            # target label index
            t_inds = [_i for _i in range(x_length) if _i != C]

            # each target label
            for i, y in zip(t_inds, ys):
                # target label
                t = torch.tensor(Xs[i], dtype=torch.long).to(device)

                # get loss
                loss += loss_fn(torch.log(y), t)

                # count accuracy
                if y.argmax() == t:
                    accuracy += 1

            """
            # each target label
            for i in range(x_length):
                # forward network
                y = model(input_X)

                # target label
                t = torch.tensor(Xs[i], dtype=torch.long).to(device)
                #t = torch.tensor(Xs[i], dtype=torch.long).to(device).view(-1, voca_num)

                # get loss
                loss += loss_fn(torch.log(y), t)

                # count accuracy
                if y.argmax() == t:
                    accuracy += 1
            """

        # loss backward
        loss.backward()

        # update weight
        optimizer.step()
        
        # get loss
        loss = loss.item() / total_len
        accuracy = accuracy / total_len

        if (ite + 1) % 10 == 0:
            print("iter :", ite+1, ",loss >>:", loss, "accuracy:", accuracy)

    torch.save(model.state_dict(), 'word2vec.pt')